In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import numpy as np
import pickle
from sklearn.metrics import mean_squared_error

In [2]:
jan_23 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
feb_23 = 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

## Question 1

- Read the data for January. How many columns are there?

In [3]:
df_01 = pd.read_parquet(jan_23)
print(f"There are {len(df_01.columns)} columns in the january yellow data from 2023")

There are 19 columns in the january yellow data from 2023


In [4]:
df_02 = pd.read_parquet(feb_23)

## Question 2

- Compute the duration variable. It should contain the duration of a ride in minutes.

- What's the standard deviation of the trips duration in January?

In [5]:
duration2 = df_02.tpep_dropoff_datetime - df_02.tpep_pickup_datetime
df_02['duration'] = duration2
df_02.duration = df_02.duration.apply( lambda td: td.total_seconds() / 60 )

In [6]:
duration1 = df_01.tpep_dropoff_datetime - df_01.tpep_pickup_datetime
df_01['duration'] = duration1
df_01.duration = df_01.duration.apply( lambda td: td.total_seconds() / 60 )

In [7]:
print(f"The standard deviation of the trips duration in january is {np.round(df_01.duration.std(),2)}.")

The standard deviation of the trips duration in january is 42.59.


## Question 3

- What fraction of the records left after you dropped the outliers?

I decide to also delete trips thata re too fast or too slow. For that I should calculate the mean speed of eahc trip.

$$ v = distance/duration$$

In [8]:
df_01['speed_mph'] = df_01['trip_distance'] / (df_01['duration'] / 60)  # Convert minutes to hours
#df_01.drop(columns=['speed_mpm'], inplace=True)  # Drop 'speed_mpm' if it exists

df_02['speed_mph'] = df_02['trip_distance'] / (df_02['duration'] / 60)  # Convert minutes to hours
#df_02.drop(columns=['speed_mpm'], inplace=True)  # Drop 'speed_mpm' if it exists


In [9]:
# manually 
df1 = df_01[(df_01.duration > 1) & (df_01.duration < 100) & (df_01.trip_distance != 0) & 
            (df_01.trip_distance < 50) & (df_01.speed_mph > 1) & (df_01.speed_mph < 240)]

df2 = df_02[(df_02.duration > 1) & (df_02.duration < 100) & (df_02.trip_distance != 0) & 
            (df_02.trip_distance < 50) & (df_02.speed_mph > 1) & (df_02.speed_mph < 240)]

In [10]:
print( f" The fraction remaining after outliers removal is {np.round((len(df1) / len(df_01)) * 100 , 1)}% for january data \n and {np.round((len(df2) /len(df_02))*100 , 1)}% for february data")

 The fraction remaining after outliers removal is 98.0% for january data 
 and 98.0% for february data


## Question 4

- One-hot encoding

In [11]:
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,speed_mph
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,...,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333,6.901186
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,...,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667,10.448549
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,...,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000,11.811765
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,...,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667,11.854419
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,...,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333,7.920000


In [14]:
# variables
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance', 'fare_amount', 'total_amount', 'speed_mph']

df1[categorical] = df_01[categorical].astype(str)
df2[categorical] = df_02[categorical].astype(str)

train_dic = df1[categorical].to_dict(orient='records')
val_dic = df2[categorical].to_dict(orient='records')

In [16]:
dv = DictVectorizer()

X_train = dv.fit_transform(train_dic)
X_val = dv.transform(val_dic)

In [17]:
target = 'duration'
y_train = df1[target].values
y_val = df2[target].values

In [25]:
print(f" Number of columns is {X_train.shape[1]} for the train matrix")

 Number of columns is 515 for the train matrix


## Question 5

- Train a plain linear regression model with default parameters, where duration is the response variable
- Calculate the RMSE of the model on the training data


In [28]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [32]:
y_pred_train = lr.predict(X_train)

mse_t = mean_squared_error(y_train, y_pred_train, squared=False)
print(mse_t)

8.213280797333441


## Question 6

In [33]:
y_pred_val = lr.predict(X_val)
mse_v = mean_squared_error(y_val, y_pred_val, squared=False)
print(mse_v)

8.470760815750761
